## Package installation

In [1]:
# !pip install pycocotools
# !pip install fiftyone

In [2]:
from pycocotools.coco import COCO
from PIL import Image

In [3]:
coco_annotation_file_path = r"""D:\xing-vimentin-dic-pipeline\src\cxa_segmentation\notebooks\labelme2coco_test_output\dataset.json"""

coco_annotation = COCO(annotation_file=coco_annotation_file_path)


loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [4]:
coco_annotation.cats
coco_annotation.anns[1]

{'iscrowd': 0,
 'image_id': 1,
 'bbox': [1137, 1309, 283, 264],
 'segmentation': [[1256,
   1308,
   1347,
   1322,
   1411,
   1375,
   1419,
   1417,
   1394,
   1486,
   1342,
   1542,
   1275,
   1572,
   1194,
   1539,
   1150,
   1486,
   1136,
   1433,
   1175,
   1364]],
 'category_id': 0,
 'id': 1,
 'area': 52403}

In [6]:
from matplotlib import pyplot as plt
import numpy as np
import os
from pathlib import Path

mask = coco_annotation.annToMask(coco_annotation.anns[3])

output_dir = "./tmp/coco_binary_aggregated_masks"
os.makedirs(output_dir, exist_ok=True)

def convert_coco_cat_to_mask(coco_annotation, cat_id, output_dir):
    """Converts a coco category to a binary mask. For each image, collect all of its annotations and aggregate them into a single mask.

    Parameters
    ----------
    coco_annotation : 
        COCO annotation object
    cat_id : 
        category IDs specified in COCO json files
    output_dir :
        Output directory to save the masks
    """
    coco_img_ids = list(coco_annotation.imgs.keys())
    id2anns = coco_annotation.imgToAnns
    for img_id, anns in id2anns.items():
        mask = None
        for ann in anns:
            if not (ann["category_id"] == cat_id):
                continue
            tmp_mask = coco_annotation.annToMask(ann)
            if mask is None:
                mask = tmp_mask
            mask = np.logical_or(mask, tmp_mask)
        # print(type(mask))
        # plt.imshow(mask)
        # plt.show()
        img = Image.fromarray(mask)
        img.save(Path(output_dir) / f"{img_id}.png")
convert_coco_cat_to_mask(coco_annotation, cat_id=0, output_dir=output_dir)